Platforma pentru experimente privind modele "low complexity" de sisnapse (in particular sinapsa comparativa) 
GIT-ul de plecare [1] ofera cod exclusiv NUMPY (nu Tensorflow sau Keras) cu avantajul ca se poate interveni la orice nivel asupra algoritmului (de ex. inlocuirea sinapsei clasice cu sinapsa comparativa. 

--------------

**To do:**  
Pentru eficientizare si posibilitatea de utilizare GPU, utilizare Cupy in loc de Numpy ( alt cod cu utilizare CUPY aici  https://github.com/radu-dogaru/LB-CNN-compact-and-fast-binary-including-very-fast-ELM ) 

Inserarea / evaluarea unor modele noi de sinapsa (in principal cea comparativa) 

Nota: pentru sinapsa comparativa in aceasta etapa trebuiesc 
a) inlocuit fisierul nn.py cu cel modificat (arhiva atasata) 
b) in directorul layers inserate cfc.py si cconv.py 
(suport sinapse comparative) 

Radu Dogaru, 1 oct. 2021 

In [1]:
#!git clone https://github.com/lpraat/numpyCNN
!git clone https://github.com/radu-dogaru/numpyCNN

Cloning into 'numpyCNN'...
remote: Enumerating objects: 65, done.
remote: Counting objects: 100% (12/12), done.
remote: Compressing objects: 100% (12/12), done.
remote: Total 65 (delta 5), reused 0 (delta 0), pack-reused 53
Unpacking objects: 100% (65/65), done.


Recomandat ca fiind "clar"


In [2]:
cd /content/numpyCNN

/content/numpyCNN


In [9]:
import numpy as np

import mnist as my_mnist
from src.activation import relu, softmax, identity, sigmoid
from src.cost import softmax_cross_entropy
from src.layers.conv import Conv
from src.layers.cconv import CConv
from src.layers.dropout import Dropout
from src.layers.fc import FullyConnected
from src.layers.cfc import CFullyConnected
from src.layers.flatten import Flatten
from src.layers.pool import Pool
from src.nn import NeuralNetwork
from src.optimizer import adam, rmsprop


In [6]:
# This cell expands the usual datasest available in Keras 
!pip install extra-keras-datasets
from extra_keras_datasets import emnist, svhn, stl10, usps

Datasets loading 

In [7]:
dataset='usps' # mnist or f-mnist or cifar10 

import numpy as np
import tensorflow as tf

dformat='channels_last'

from keras.datasets import mnist, cifar10, cifar100, fashion_mnist

if dataset=='mnist':
      (x_train, y_train), (x_test, y_test) = mnist.load_data() # incarca date nescalate 
elif  dataset=='cifar10': 
      (x_train, y_train), (x_test, y_test) = cifar10.load_data() # incarca date nescalate 
      y_test=y_test.T
elif  dataset=='cifar100': 
      (x_train, y_train), (x_test, y_test) = cifar100.load_data() # incarca date nescalate 
elif dataset=='f-mnist':
      (x_train, y_train), (x_test, y_test) =  fashion_mnist.load_data()
elif dataset=='usps':
    (x_train, y_train), (x_test, y_test) =  usps.load_data()
    y_train=y_train.astype('int32')
    x_train=256*x_train; x_test=256*x_test

if (np.ndim(x_train)==3):   # E.g.  MNIST or F-MNIST  
      x_train=np.reshape(x_train, [np.shape(x_train)[0],np.shape(x_train)[1],np.shape(x_train)[2], 1]) 
      x_test=np.reshape(x_test, [np.shape(x_test)[0],np.shape(x_test)[1],np.shape(x_test)[2], 1] ) 
    # place a  1 in the end to keep it compatible with kernel in conv2d 
    # scaling in ([0,1])
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /=255 
num_classes=np.max(y_train)+1
input_shape=np.shape(x_train)[1:4]
y_train = tf.keras.utils.to_categorical(y_train, num_classes)
#y_test = tf.keras.utils.to_categorical(y_test, num_classes)

# y_train in format categoric si y_test in format normal 

INFO:root:Loading dataset = usps


1843200/1831726 [==============================] - 1s 0us/step


Defining the CNN model (based on existent layers) 

In [45]:
cnn = NeuralNetwork(
        input_dim=input_shape,
        layers=[
            CConv(3, 1, 10, activation=relu),
            Pool(2, 2, 'max'),
            #CConv(3, 1, 8, activation=relu),

            #Conv(3, 1, 4, activation=relu),
            #Conv(3, 1, 12, activation=relu),
            #Conv(3, 1, 32, activation=relu),
            #Conv(5, 1, 32, activation=relu),
            #Pool(2, 2, 'max'),
            #Conv(3, 1, 8, activation=relu),
            #Pool(2, 2, 'max'),
            #Conv(3, 1, 8, activation=relu),
            #Pool(2, 2, 'max'),
            #Conv(3, 1, 64, activation=relu),
            #Pool(2, 2, 'max'),
            
            #Dropout(0.75),
            Flatten(),
            #FullyConnected(500, relu),
            #Dropout(0.9),
            #CFullyConnected(10, softmax),
            FullyConnected(10, softmax),
        ],
        cost_function=softmax_cross_entropy,
        optimizer=rmsprop
    )


Training the model defined above 

In [46]:
cnn.train(x_train, y_train,
              mini_batch_size=100,
              learning_rate=0.01,
              num_epochs=20,
              validation_data=(x_test, y_test))

Started training [batch_size=100, learning_rate=0.01]
Epoch 1
Progress 100.0%
Cost after epoch 1: 1.1397604990567676
Computing accuracy on validation set...
Accuracy on validation set:  0.9018435475834579
Latentza pe set test:  0.42645433499080126 mili-secunde
Durata epoca:  7.483052730560303  secunde
Epoch 2
Progress 100.0%
Cost after epoch 2: 0.17880336769001648
Computing accuracy on validation set...
Accuracy on validation set:  0.9277528649725959
Latentza pe set test:  0.416891932428091 mili-secunde
Durata epoca:  7.313705205917358  secunde
Epoch 3
Progress 100.0%
Cost after epoch 3: 0.10103575526254227
Computing accuracy on validation set...
Accuracy on validation set:  0.8963627304434479
Latentza pe set test:  0.4173325375650555 mili-secunde
Durata epoca:  7.381204128265381  secunde
Epoch 4
Progress 100.0%
Cost after epoch 4: 0.05762590552930371
Computing accuracy on validation set...
Accuracy on validation set:  0.9392127553562531
Latentza pe set test:  0.42174642930650924 mili-

Obs: 

With CPU (GPU is not supported in NUMPY) the training lasts long 

Code can be used as a starting template for further improvements. It may be also conveninet for TinyML implements (low complexity microcontrollers supporting Python / Numpy) 

R. Dogaru, Oct. 2021